# Multiple Linear Regression with SKLearn

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('1.02. Multiple linear regression.csv')
data.head()

,SAT,GPA,"Rand 1,2,3"
0,1714,2.40,1
1,1664,2.52,3
2,1760,2.54,3
3,1685,2.74,3
4,1693,2.83,2


In [3]:
data.describe()

,SAT,GPA,"Rand 1,2,3"
count,84.000000,84.000000,84.000000
mean,1845.273810,3.330238,2.059524
std,104.530661,0.271617,0.855192
min,1634.000000,2.400000,1.000000
25%,1772.000000,3.190000,1.000000
50%,1846.000000,3.380000,2.000000
75%,1934.000000,3.502500,3.000000
max,2050.000000,3.810000,3.000000


In [4]:
x = data[['SAT', 'Rand 1,2,3']]
y = data['GPA']

In [5]:
reg = LinearRegression()
reg.fit(x, y)   # We didn't have to reshape the independent variables since the array is already in more than one dimension, since we are using more than one variable
# Hence, we can see that sklearn is optimised for multiple linear regression first

LinearRegression()

In [6]:
reg.coef_   # This gives the output in the same order as fed. We know that SAT comes first in x, the first coefficient belongs to it

array([ 0.00165354, -0.00826982])

In [7]:
reg.intercept_

0.29603261264909575

In [8]:
reg.score(x, y)

0.4066811952814282

### Now, the adjusted R squared is the better method to decide whether the algorithm works well (for multiple linear regression).
Unfortunately, there is no function in sklearn for the adjusted r-squared, so we would have to calculate it on our own

The formula for Adjusted R^2:

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mstyle displaystyle="true" scriptlevel="0">
    <mrow data-mjx-texclass="ORD">
      <mtable rowspacing=".5em" columnspacing="1em" displaystyle="true">
        <mtr>
          <mtd>
            <msubsup>
              <mi>R</mi>
              <mrow data-mjx-texclass="ORD">
                <mi>a</mi>
                <mi>d</mi>
                <mi>j</mi>
                <mo>.</mo>
              </mrow>
              <mn>2</mn>
            </msubsup>
            <mo>=</mo>
            <mn>1</mn>
            <mo>&#x2212;</mo>
            <mo stretchy="false">(</mo>
            <mn>1</mn>
            <mo>&#x2212;</mo>
            <msup>
              <mi>R</mi>
              <mn>2</mn>
            </msup>
            <mo stretchy="false">)</mo>
            <mo>&#x2217;</mo>
            <mfrac>
              <mrow>
                <mi>n</mi>
                <mo>&#x2212;</mo>
                <mn>1</mn>
              </mrow>
              <mrow>
                <mi>n</mi>
                <mo>&#x2212;</mo>
                <mi>p</mi>
                <mo>&#x2212;</mo>
                <mn>1</mn>
              </mrow>
            </mfrac>
          </mtd>
        </mtr>
      </mtable>
    </mrow>
  </mstyle>
</math>

Where n is the sample size and p is the number of predictors (the independent variables)

In [9]:
n = x.shape[0]
p = x.shape[1]
adjustedRSquared = 1 - (1 - reg.score(x, y)) * ( (n - 1)/(n-p-1) )
adjustedRSquared

0.39203134825134

## Now we can optimise the model so that it doesn't have too many features
Feature selection simplifies models, improves speeds and prevents a series of unwanted issues arising from having too many features.

Earlier, we did this by checking the p-values behind each feature to determine its necessity. However, sklearn doesn't have that feature.
So instead, we will have to go with f_regression.

F-Regression creates simple linear regressions for each feature separately.

In [10]:
from sklearn.feature_selection import f_regression

In [11]:
f_regression(x, y)  # The first array of the output is the F stats for each variable, while the second array are the p-values.

(array([56.04804786,  0.17558437]), array([7.19951844e-11, 6.76291372e-01]))

In [12]:
p_values = f_regression(x, y)[1]
p_values

array([7.19951844e-11, 6.76291372e-01])

In [13]:
p_values.round(3)

array([0.   , 0.676])

## This means that SAT is significant, while Rand 1,2,3 is useless

In [14]:
reg_summary = pd.DataFrame(data=["SAT", 'Rand 1,2,3'], columns=['Features'])
reg_summary

,Features
0,SAT
1,"Rand 1,2,3"


In [15]:
reg_summary['Coefficients'] = reg.coef_
reg_summary['p-values'] = p_values.round(3)
reg_summary

,Features,Coefficients,p-values
0,SAT,0.001654,0.000
1,"Rand 1,2,3",-0.008270,0.676
